## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-04-17-08-16 +0000,nypost,Olympics boss Casey Wasserman ‘is a distractio...,https://nypost.com/2026/02/04/us-news/casey-wa...
1,2026-02-04-17-07-32 +0000,bbc,Restaurant boss 'devastated' at having to close,https://www.bbc.com/news/articles/cwyw77k3ze6o...
2,2026-02-04-17-04-39 +0000,nyt,Washington Post Begins Cutting More Than 300 Jobs,https://www.nytimes.com/2026/02/04/business/me...
3,2026-02-04-17-04-00 +0000,wsj,A key Republican senator said he doesn’t belie...,https://www.wsj.com/economy/central-banking/to...
4,2026-02-04-17-02-13 +0000,nyt,Russia and Ukraine Resume Talks After a Huge A...,https://www.nytimes.com/2026/02/04/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2412/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
193,trump,39
7,epstein,26
56,new,24
273,house,20
96,you,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
271,2026-02-04-00-00-00 +0000,wsj,Stephen Miller called Alex Pretti a “domestic ...,https://www.wsj.com/politics/policy/ice-stephe...,119
256,2026-02-04-01-07-00 +0000,wsj,Fed governor Stephen Miran said he resigned fr...,https://www.wsj.com/economy/central-banking/st...,114
288,2026-02-03-22-56-00 +0000,wsj,The Republican-led House passed funding legisl...,https://www.wsj.com/politics/policy/government...,109
308,2026-02-03-22-17-00 +0000,wsj,President Trump shared a rendering Tuesday of ...,https://www.wsj.com/politics/policy/trump-whit...,103
279,2026-02-03-23-21-07 +0000,bbc,Trump hails White House talks with 'terrific' ...,https://www.bbc.com/news/articles/c4g01vlwqp5o...,89


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
271,119,2026-02-04-00-00-00 +0000,wsj,Stephen Miller called Alex Pretti a “domestic ...,https://www.wsj.com/politics/policy/ice-stephe...
21,80,2026-02-04-16-39-42 +0000,nypost,New Epstein emails about top Goldman lawyer Ka...,https://nypost.com/2026/02/04/business/epstein...
256,70,2026-02-04-01-07-00 +0000,wsj,Fed governor Stephen Miran said he resigned fr...,https://www.wsj.com/economy/central-banking/st...
130,59,2026-02-04-12-35-16 +0000,nyt,Savannah Guthrie to Skip Olympics as Search fo...,https://www.nytimes.com/2026/02/03/us/savannah...
288,50,2026-02-03-22-56-00 +0000,wsj,The Republican-led House passed funding legisl...,https://www.wsj.com/politics/policy/government...
264,49,2026-02-04-00-31-15 +0000,nypost,Netflix co-CEO grilled by senators as Warner B...,https://nypost.com/2026/02/03/business/netflix...
89,49,2026-02-04-14-35-16 +0000,startribune,"Minnesota school districts, teachers union sue...",https://www.startribune.com/minnesota-school-d...
259,46,2026-02-04-00-54-23 +0000,cbc,French police raid X offices as they investiga...,https://www.cbc.ca/news/world/french-police-ra...
383,46,2026-02-03-18-44-52 +0000,nypost,Putin breaks Trump’s cease-fire after just 4 d...,https://nypost.com/2026/02/03/world-news/putin...
366,43,2026-02-03-19-41-36 +0000,nypost,Jeffrey Epstein asked woman to buy Snow White ...,https://nypost.com/2026/02/03/business/jeffrey...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
